# ▶️ Initial Setup

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
ROOT_DIR = "../data/osha/"

In [2]:
import os, sys
sys.path.append("src/")
sys.path.append("../src/")

In [3]:
# Import modules for LLM finetuning and evaluation
from utils import LocalModelArguments, LocalPLM
import preprocess as pre
import evaluate as ev

In [ ]:
import model_prompts as prompts
from evaluate import EvaluationConfig

def evaluate_model(configurations, model, label_names, eval_dataset):
    results = []
    for config in configurations:
        result = ev.evaluate(
            model=model, label_names=label_names,
            eval_dataset=dataset['test'], eval_config=config
        )
        results.append(result)
    return results

def save_results(results):
    for result in results:
        result.save(os.path.join(ROOT_DIR, "results")) # Saves to "data/osha/results/<EvaluationConfig.name>"

# ▶️ Load and Preprocess OSHA Injuries dataset

In [ ]:
import pandas as pd

data = pd.read_csv(
    os.path.join(ROOT_DIR, "datasets/January2015toJuly2024.csv"),
    low_memory=False)

In [ ]:
input_features = "Final Narrative"
output_labels = "NatureTitle"

## Imbalanced

In [ ]:
# Remove all classes with less than 50 samples
min_class_size = 50
while True:
    class_counts = imbalanced_data[output_labels].value_counts()
    classes_to_remove = class_counts.where(lambda x: x <= min_class_size).dropna().keys().to_list()
    imbalanced_data = imbalanced_data[~imbalanced_data[output_labels].isin(classes_to_remove)]
    
    if imbalanced_data[output_labels].value_counts().where(lambda x: x <= min_class_size).dropna().size == 0:
        break

In [ ]:
imbalanced_dataset = pre.create_dataset_from_dataframe(imbalanced_data, input_features, output_labels, test_size=0, encode_labels=False)
imbalanced_dataset = imbalanced_dataset.to_pandas()
imbalanced_dataset.to_csv(os.path.join(ROOT_DIR, "datasets/imbalanced.csv"), index=False)

## Balanced

In [ ]:
# Convert DataFrame into a Dataset
balanced_dataset = pre.create_dataset_from_dataframe(data, input_features, output_labels, test_size=0)

In [ ]:
# Select items from the 10 most common classes
balanced_dataset = pre.select_top_n_classes(balanced_dataset, n=10, label_columns=output_labels)

In [ ]:
balanced_dataset = pre.undersample_dataset(balanced_dataset, label_columns="NatureTitle", ratio=1)

In [ ]:
balanced_dataset, labels = pre.class_decode_column(balanced_dataset, "NatureTitle")

In [ ]:
balanced_dataset = balanced_dataset.to_pandas()

In [ ]:
balanced_dataset.to_csv(os.path.join(ROOT_DIR, "datasets/balanced.csv"), index=False)

In [ ]:
# Preprocess the dataset into a form usable for supervised finetuning

dataset, label_names = pre.load_dataset(
    os.path.join(ROOT_DIR, "datasets/balanced.csv"),
    input_features,
    output_labels,
    test_size=0.2)

In [ ]:
# Reduce the size of the dataset for testing purposes
dataset['train'] = dataset['train'].shard(5, 0) # First 20% of the train dataset
dataset['test'] = dataset['test'].shard(10, 0) # First 10% of the test dataset

# ▶️ Load Baseline LLM

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "Qwen/Qwen2.5-7B-Instruct",
    cuda_devices = "1",
    use_4bit_quantization = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    use_nested_quant = True,
    use_reentrant = True
)

model = LocalPLM(args)

# ▶️ Evaluate Baseline LLM

In [ ]:
# Configurations for the baseline LLM
baseline_configurations = [
    EvaluationConfig(
        technique_name="Zero-shot Multi-task",
        prompt=prompts.OSHA["MULTI_TASK"],
        max_tokens=40
    )
]

In [ ]:
baseline_results = evaluate_model(baseline_configurations, model=model, label_names=label_names, eval_dataset=dataset['test'])

In [ ]:
save_results(baseline_results)

# ▶️ Finetune LLM

In [ ]:
FINETUNED_LLM_PATH = os.path.join(ROOT_DIR, "models/Qwen2.5-FT-OSHA-Injuries-Multiclass-2")

LORA_RANK_DIMENSION = 6 # the rank of the adapter, the lower the fewer parameters you'll need to train. (smaller = more compression)
LORA_ALPHA = 8 # this is the scaling factor for LoRA layers (higher = stronger adaptation)
LORA_DROPOUT = 0.05 # dropout probability for LoRA layers (helps prevent overfitting)
MAX_SEQ_LENGTH = 64
EPOCHS=1
LEARNING_RATE=2e-4

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=LORA_RANK_DIMENSION,
    lora_alpha=LORA_ALPHA,
    bias="none",
    lora_dropout=LORA_DROPOUT,
    task_type="CAUSAL_LM"
)

In [ ]:
from trl import SFTConfig, SFTTrainer

sft_config = SFTConfig(
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant': False},
    gradient_accumulation_steps=1,
    per_device_train_batch_size=16,
    auto_find_batch_size=True,
    
    max_seq_length=MAX_SEQ_LENGTH,
    packing=True,
    
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    optim='adamw_torch_fused',
    warmup_ratio=0.03,
    lr_scheduler_type="constant", 
    
    logging_steps=10,
    logging_dir='./logs',
    output_dir=FINETUNED_LLM_PATH,
    report_to='none'
)

In [ ]:
trainer, history = model.finetune(
    train_dataset=dataset['train'],
    lora_config=lora_config, sft_config=sft_config
)

In [ ]:
history.to_csv(os.path.join(ROOT_DIR, "results/multi-class/loss_history.csv"), index=False) # Save the training history

In [ ]:
# Plot the training history and save the plot
import math
from matplotlib import pyplot as plt

plt.rcParams['figure.dpi'] = 150
plt.plot(history.set_index("Step"))
plt.xlabel("Epoch")
plt.ylabel("Training Loss")

loss_max = math.ceil(history['Training Loss'].max())
plt.ylim([0, loss_max])

plt.title("Fine-tuning Training History")

path = os.path.join(ROOT_DIR, "results/multi-class/loss_history.png")
plt.savefig( path, dpi=200, bbox_inches='tight' )

# ▶️ Load Finetuned LLM

In [ ]:
# Unload the baseline model if it exists, otherwise we will probably get an OOM exception
import gc, torch

if "model" in locals(): del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "models/Qwen2.5-FT-OSHA-Injuries-Multiclass-2",
    cuda_devices = "1",
    use_4bit_quantization = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = "float16",
    use_nested_quant = False
)

model = LocalPLM(args)

# ▶️ Evaluate Finetuned LLM

In [ ]:
finetuned_configurations = [
    EvaluationConfig(
        technique_name="Fine-tuned Multi-task",
        prompt=None,
        max_tokens=40
    )
]

In [ ]:
finetuned_results = evaluate_model(finetuned_configurations, model=model, label_names=label_names, eval_dataset=dataset['test'])

In [ ]:
save_results(finetuned_results)